# Scanning Session Ingestion

## Setup

### Connect to the database

If you are don't have your login information, contact the administrator.

Using local config file (see [01_pipeline](./01_pipeline.ipynb)):

In [1]:
import os
# change to the upper level folder to detect dj_local_conf.json
if os.path.basename(os.getcwd())=='notebooks': os.chdir('..')
assert os.path.basename(os.getcwd())=='adamacs', ("Please move to the main directory")
import datajoint as dj; dj.conn()

from adamacs.pipeline import subject, session, surgery, scan, equipment
from adamacs import utility
from adamacs.ingest import session as isess
sub, lab, protocol, line, mutation, user, project, subject_genotype, subject_death = (
    subject.Subject(), subject.Lab(), subject.Protocol(), subject.Line(), 
    subject.Mutation(), subject.User(), subject.Project(), subject.SubjectGenotype(), 
    subject.SubjectDeath())

[2022-12-16 13:33:49,442][INFO]: Connecting tobiasr@172.26.128.53:3306
[2022-12-16 13:33:49,494][INFO]: Connected tobiasr@172.26.128.53:3306


Loading DLC 2.2.1.1...


/Users/trose/opt/miniconda3/envs/datajoint/lib/python3.8/site-packages/statsmodels/compat/pandas.py:65: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import Int64Index as NumericIndex


DLC loaded in light mode; you cannot use any GUI (labeling, relabeling and standalone GUI)


In [ ]:
# subject.Subject.drop()
# subject.Mutation.drop()
# subject.Protocol.drop()
# subject.Line.drop()
# subject.SubjectGenotype.drop()
# equipment.Equipment.drop()
# scan.Scan.drop()
# session.Session.drop()
imaging.Processing.drop()

In [ ]:
scan.Scan.describe()

In [ ]:
dj.Diagram(subject) + dj.Diagram(protocol)

Manual entry:

In [ ]:
# Manual Entry
import datajoint as dj; import getpass
dj.config['database.host'] = '172.26.128.53'        # Put the server name between these apostrophe
dj.config['database.user'] = 'danielmk'             # Put your user name between these apostrophe
dj.config['database.password'] = getpass.getpass()  # Put your password in the prompt
dj.conn()

from adamacs.pipeline import subject, session, surgery, scan, equipment
from adamacs import utility
from adamacs.ingest import session as isess
sub, lab, protocol, line, mutation, user, project, subject_genotype, subject_death = (
    subject.Subject(), subject.Lab(), subject.Protocol(), subject.Line(), 
    subject.Mutation(), subject.User(), subject.Project(), subject.SubjectGenotype(), 
    subject.SubjectDeath())


## Ingesting scan

Ingest all scans associated with a given session ID.

In [2]:
# isess.ingest_session_scan('sess9FB2LN5C', verbose=True)
isess.ingest_session_scan('sess9FF6TL96', verbose=True) #TR: needs to be SessioID ("sess...")

Number of scans found: 1
[['scan9FF6TL96']]
['JJ_ROS-1438_2022-11-22_scan9FF6TL96_sess9FF6TL96']


/var/folders/p5/fxz77v2s4n94wnrrm2lycb780000gn/T/ipykernel_16270/134453612.py:2: UserWarning: 
Skipped existing session row: ('sess9FF6TL96', <module 'adamacs.schemas.subject' from '/Users/trose/Documents/GitHub/troselab/adamacs/adamacs/schemas/subject.py'>, '2022-11-22')
  isess.ingest_session_scan('sess9FF6TL96', verbose=True) #TR: needs to be SessioID ("sess...")
/var/folders/p5/fxz77v2s4n94wnrrm2lycb780000gn/T/ipykernel_16270/134453612.py:2: UserWarning: 
Skipped existing SessionDirectory: ('sess9FF6TL96', PosixPath('/Users/trose/Data/JJ_ROS-1438_2022-11-22_scan9FF6TL96_sess9FF6TL96'))
  isess.ingest_session_scan('sess9FF6TL96', verbose=True) #TR: needs to be SessioID ("sess...")
/var/folders/p5/fxz77v2s4n94wnrrm2lycb780000gn/T/ipykernel_16270/134453612.py:2: UserWarning: 
Skipped existing SessionUser row: ('sess9FF6TL96', 6)
  isess.ingest_session_scan('sess9FF6TL96', verbose=True) #TR: needs to be SessioID ("sess...")
/var/folders/p5/fxz77v2s4n94wnrrm2lycb780000gn/T/ipykernel_162

In [4]:
# isess.ingest_session_scan('sess9FB2LN5C', verbose=True)
isess.ingest_session_scan('sess9FF6U114', verbose=True) #TR: needs to be SessioID ("sess...")

Number of scans found: 1
[['scan9FF6U114']]
['JJ_ROS-1438_2022-11-22_scan9FF6U114_sess9FF6U114']


In [5]:
session.Session b* session.SessionDirectory

session_id,subject PyRat import uses this for earmark value,session_datetime,session_dir Path to the data directory for a session,user_id
sess9FF6TL96,ROS-1438,2022-11-22 00:00:00,/Users/trose/Data/JJ_ROS-1438_2022-11-22_scan9FF6TL96_sess9FF6TL96,1
sess9FF6U114,ROS-1438,2022-11-22 00:00:00,/Users/trose/Data/JJ_ROS-1438_2022-11-22_scan9FF6U114_sess9FF6U114,6


In [8]:
session.SessionDirectory()

session_id,session_dir Path to the data directory for a session,user_id
sess9FF6TL96,/Users/trose/Data/JJ_ROS-1438_2022-11-22_scan9FF6TL96_sess9FF6TL96,1
sess9FF6U114,/Users/trose/Data/JJ_ROS-1438_2022-11-22_scan9FF6U114_sess9FF6U114,6


In [ ]:
key='scan9FF6TL96'
(scan.Scan & f'scan_id=\"{key}\"')

Simple user look-up to get user_id from shorthand

In [13]:
key='JJ'
query = (subject.User & f'initials=\"{key}\"')
id = query.fetch("user_id")[0]
print(key + ' = ' + str(id))

TR = 1


In [ ]:
query

In [ ]:
query = subject.User() & 'initial = "WEZ-8701"'

In [ ]:
subject.User(.

##### Some placeholders for equipment and location during development

In [ ]:
scan.ScanInfo()

In [ ]:
scan.Scan()

In [ ]:
scan.Scan * scan.ScanInfo

Note the relative path below:

In [ ]:
#temporary step - insert placeholder values
equipment_placeholder = "Equipment"
location_placeholder = "Location"
from adamacs.pipeline import Equipment, Location
Equipment.insert1({'scanner' : equipment_placeholder}, skip_duplicates=True)
Location.insert1({'anatomical_location': location_placeholder}, skip_duplicates=True) 

In [ ]:
Equipment()
